# Mapa em formato de `Marcadores com várias layers`

## Carregando os dados

In [6]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados do INPE
url_diario = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/'

# Loop nos dias
df = pd.DataFrame()
anos, meses, dias, ano_mes_dia = [], [], [], []
for i in range(8):

    # gera as datas
    data = datetime.datetime.now() - timedelta(days=i)
    
    ano, mes, dia = date.strftime(data,'%Y'), date.strftime(data,'%m'), date.strftime(data,'%d')
    ano_mes_dia.append(f'{ano}-{mes}-{dia}')     
    anos.append(ano)
    meses.append(mes)
    dias.append(dia)

    # leitura dos dados
    df0 = pd.read_csv(f'{url_diario}focos_diario_br_{ano}{mes}{dia}.csv', usecols=['data_hora_gmt','lat','lon','satelite','municipio','estado'])

    # junta a tabela que foi lida com a anterior
    df = pd.concat([df, df0], ignore_index=True)

# ordena as datas
ano_mes_dia=sorted(ano_mes_dia)

# seleciona os focos de Minas Gerais e do Satélite AQUA
df_mg = df[ (df['estado']=='MINAS GERAIS') & (df['satelite']=='AQUA_M-T') ]

# transforma a data em índice
df_mg['data_hora_gmt'] = pd.to_datetime(df_mg['data_hora_gmt']) # transformar a coluna 'acq_date' em formato datetime
#df_mg.set_index('data_hora_gmt', inplace=True) # definir a coluna 'acq_date' como índice do DataFrame

# ordena o dataframe
#df_mg.sort_index(axis=0, inplace=True)
df_mg.sort_values(by=['data_hora_gmt'], inplace=True)

# cria a coluna com o ano, mês e dia
df_mg['Data'] = df_mg['data_hora_gmt'].apply(lambda x: x.strftime('%Y-%m-%d'))

# elimina algumas colunas
df_mg.drop(columns=['data_hora_gmt'], inplace=True)

# renomeia os nomes das colunas
df_mg.rename(columns={'lat': 'Latitude', 'lon': 'Longitude', 'satelite': 'Satélite', 'municipio': 'Município','estado': 'Estado'}, inplace=True)

# salva arquivo CSV
df_mg.to_csv('focos_diario.csv', index=False)

# salva arquivo CSV para cada dia
for dia in ano_mes_dia: df_mg[ df_mg['Data'] == dia].to_csv(f'focos_diario_{dia}.csv', index=False)

# mostra os dados
df_mg

CPU times: total: 219 ms
Wall time: 2.34 s


,Latitude,Longitude,Satélite,Município,Estado,Data
57176,-20.34982,-42.36221,AQUA_M-T,MATIPÓ,MINAS GERAIS,2024-06-30
57198,-18.63917,-46.50724,AQUA_M-T,PATOS DE MINAS,MINAS GERAIS,2024-06-30
57197,-18.64051,-46.51659,AQUA_M-T,PATOS DE MINAS,MINAS GERAIS,2024-06-30
57196,-18.77378,-46.68073,AQUA_M-T,PATOS DE MINAS,MINAS GERAIS,2024-06-30
57195,-18.77512,-46.69010,AQUA_M-T,PATOS DE MINAS,MINAS GERAIS,2024-06-30
...,...,...,...,...,...,...
6601,-20.64316,-43.19471,AQUA_M-T,PIRANGA,MINAS GERAIS,2024-07-06
6607,-15.66480,-41.61570,AQUA_M-T,ÁGUAS VERMELHAS,MINAS GERAIS,2024-07-06
6606,-15.71770,-41.02919,AQUA_M-T,DIVISÓPOLIS,MINAS GERAIS,2024-07-06
8949,-20.42378,-46.64646,AQUA_M-T,DELFINÓPOLIS,MINAS GERAIS,2024-07-06


## Plota mapa

In [7]:
# importa biblioteca
import leafmap

# configuração do mapa
Map = leafmap.Map(center=(-18, -40), zoom=6.3, layers_control=True)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# define a quantidade de cores baseado na quantitade de dais com focos
cores = ['lightgray', 'gray', 'blue', 'darkblue', 'green', 'darkgreen', 'red', 'darkred'] 

# plota mapa
for i, dia in enumerate(ano_mes_dia):

    qte_focos = df=pd.read_csv(f'focos_diario_{dia}.csv').shape[0]

    if qte_focos != 0:
        Map.add_points_from_xy(f'focos_diario_{dia}.csv',
                               x="Longitude",
                               y="Latitude",
                               color_column="Data",      
                               marker_colors=[cores[i]],
                               icon_names=['fire'],
                               spin=True,
                               add_legend=False,
                               layer_name=f'{dia} = {qte_focos}')

# adiciona contorno de MG
#shp_mg = 'https://github.com/giuliano-macedo/geodata-br-states/blob/main/geojson/br_states/br_mg.json'
#Map.add_geojson(shp_mg, layer_name="Munícipios de MG")

# adiciona contorno dos municípios de MG
shp_mg = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json'
Map.add_geojson(shp_mg, layer_name="Munícipios de MG")

# exibe o mapa
Map

Map(center=[-18, -40], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [8]:
# salva imagem para HTML
Map.to_html("map_marcadores_varias_layers.html")